<a href="https://colab.research.google.com/github/digdemorhan/Keras_With_Examples_For_Deep_Learning/blob/main/KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Keras Tuner'ın Colab ortamına eklenmesi
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 6.2 MB/s eta 0:00:00


In [3]:
#Keras Tuner'ın kullanılmak üzere import edilmesi
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras

In [4]:
#Veri setinin yüklenmesi
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


**Eğitim veri seti 60.000 resimden, test veri seti 10.000 resimden oluşmaktadır.**

In [5]:
#Piksel ayarı ve normalizasyonu
img_train = img_train.astype("float32") / 255.0
img_test = img_test.astype("float32") / 255.0

In [6]:
#Model kurma
def build_model(hp):
  n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
  n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
  learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
  optimizer = hp.Choice("optimizer", values=["sgd","adam"])
  if optimizer == "sgd":
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
  else:
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten())
  for _ in range(n_hidden):
    model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
  model.add(tf.keras.layers.Dense(10, activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
  return model

In [8]:
#Keras Tuner Stratejisi
random_search_tuner = kt.RandomSearch(
    build_model, #model
    objective="val_accuracy", #metrik
    max_trials=5, #kaç deney yapılacak
    overwrite=True, #eğitim başladığında herhangi bir model varsa üzerine eğitilen model yazılacak
    directory="my_fashion_mnist", #modelin kaydedileceği dizin
    project_name="my_rnd_search", #proje adı
    seed = 42 #rastgelelik sabiti
)

In [9]:
#Model eğitimi
random_search_tuner.search(img_train, label_train,
                           epochs=10,
                           validation_split=0.2,
                           )

Trial 5 Complete [00h 00m 49s]
val_accuracy: 0.749750018119812

Best val_accuracy So Far: 0.8577499985694885
Total elapsed time: 00h 03m 58s


In [10]:
#En iyi modellerin bir değişkene atanması
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
#En iyi hiperparametre kombinasyonu
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)

In [12]:
#En iyi modelin hiperparametreleri
top3_params[0].values

{'n_hidden': 8,
 'n_neurons': 37,
 'learning_rate': 0.008547485565344062,
 'optimizer': 'sgd'}

In [13]:
#En iyi modelin metrikleri
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizer: sgd
Score: 0.8577499985694885


In [14]:
best_model.fit(img_train, label_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8617 - loss: 0.3800
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8688 - loss: 0.3580
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8728 - loss: 0.3499
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8740 - loss: 0.3439
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8789 - loss: 0.3341


**Modelin performansından memnunsak validation verisi kullanmadan modeli birkaç epoch daha eğitebiliriz.**

In [15]:
#Tüm veri seti üzerinde eğitilen modelin test verileri üzerinde değerlendirilmesi
test_loss, test_accuracy = best_model.evaluate(img_test, label_test)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8680 - loss: 0.3773
Test loss: 0.3812558352947235
Test accuracy: 0.8650000095367432


**Modelin test verileri üzerindeki doğruluğu eğitim verileri üzerindeki doğruluğa yakın olduğu için modelde ezberleme durumu yoktur.**